In [1]:
import numpy as np
import copy
import time

# Responsible for  managing a sudoku state through its fields and methods. 
# To create an object, a 9x9 numpy array should be provided through the constructor. 
# Optionally, an array of possible values and an array of final values can be provided as well. 
# The field `possible_values` holds the domain of each variable and the field `final_values` holds the values that have already been assigned to variables.
class sudoku_state:
    
    def __init__(self, arr, possible_values = None, final_values = None):
        
        self.arr = arr
        
        if possible_values is None:
                self.possible_values = self.create_possible_values()
        else:
            self.possible_values = possible_values
            
        if final_values is None:
            self.final_values = np.full(shape = (9,9), fill_value = 0, dtype=np.int)
        else:
            self.final_values = final_values
    
    
    # Creates the array possible_values if it is not provided through the constructor.
    def create_possible_values(self):

        possible_values = [1]*81
        
        self.arr = np.reshape (self.arr,81)
        
        for i in range(81):
            if self.arr[i] == 0:
                possible_values[i] = [k for k in range(1, 10)]
            else:
                possible_values[i] = [self.arr[i]]

        possible_values = np.array(possible_values).reshape(9,9)

        return possible_values

    # Method for aplpying the constraints and strategies.               
    def constraints(self):

        for row,l in enumerate(self. possible_values):
            
            for column,val in enumerate(l):
                
                if len(val) > 1:
                    self.no_alternative(row,column,val)

                if len(val) == 1 and self.final_values[row,column] != val[0]:
                    self.basic_constraints(row,column,val)

                if len(val) == 2:
                    self.naked_twins(row,column,val)
    
    
    def generate_box(self,row,column):
        
        box_row_start = (row//3)*3
        box_col_start = (column//3)*3
        box = self. possible_values[box_row_start:box_row_start + 3, box_col_start:box_col_start+3].reshape(9)
        
        return box
    
    # Updates the possible values in the peer cells according to the rules of sudoku puzzles.
    def update (self,row,column,value):
        
        box = self.generate_box(row,column)

        for i in range(9):
                
            if value in self.possible_values[row][i] and i != column :
                self. possible_values[row][i].remove(value)

            if value in self.possible_values[i][column] and i != row:
                self.possible_values[i][column].remove(value)

            if value in box[i] and i != row%3 * 3 + column%3:
                box[i].remove(value)
                
    # If a possible value of a cell does not occur as a possible value in some of the peer units, it assigns that value to the cell and updates the possible values of peers.
    def no_alternative(self,row,column,values):

        peers = self.find_peers(row,column)
        peers_row = [i for j in peers[0] for i in j]
        peers_column = [k for l in peers[1] for k in l]
        peers_box = [i for j in peers[2] for i in j] 
        
        for k in values:
            
            if k not in peers_row or k not in peers_column or k not in peers_box:
                self.possible_values[row][column] = [k]
                self.update(row,column,k)
                self.final_values[row][column] = k

    # Rules of sudoku puzzles.
    def basic_constraints(self,row,column,values):

        self.final_values[row,column] = values[0]
        self.update(row,column,values[0])
    
    # If two peer cells have the same domain and this is of length two it removes those values from the domains of the other peer cells.
    def naked_twins(self,row,column,values): 

        box = self.generate_box(row,column)

        peers = self.find_peers(row,column)

        occurances_row = self.count_occurances(peers[0], values)
        occurances_column = self.count_occurances(peers[1], values)
        occurances_box = self.count_occurances(peers[2], values)

        for j in range(2):    

            if occurances_row == 1:
                for i in range(9):
                    if values[j] in self.possible_values[row][i] and values != self.possible_values[row][i]:
                        self. possible_values[row][i].remove(values[j])                        

            if occurances_column == 1:
                for i in range(9):
                    if values[j] in self.possible_values[i][column] and values != self.possible_values[i][column]:
                        self.possible_values[i][column].remove(values[j])   

            if occurances_box == 1:
                for i in range(9):
                    if values[j] in box[i] and values != box[i]:
                        box[i].remove(values[j])

    # Helper method to count occurances of an element in an array.
    def count_occurances(self,arr,x):
        
        count = 0
        for i in arr:
            if type(x) == list:
                if list(i) == x:
                    count+=1
            else:
                if x == i:
                    count+=1
        return count

    # Finds the possible values of peers of a given cell.
    def find_peers(self,row,column):

        peers_row = np.concatenate((self.possible_values[row][:column], self.possible_values[row][column+1:]),axis = 0)
        peers_column = np.array([self. possible_values[i][column] for i in range(9) if i != row])

        box = self.generate_box(row,column)
        box = np.delete(box,row%3 * 3 + column%3)

        return peers_row, peers_column, box

    # Returns the possible values of peers of a cell alltogether, categorised by row,column and 3x3 box (i.e without inner lists for each cell).
    def peers_per_unit(self,row,column):
        
        peers = self.find_peers(row,column)
        peers_row = [i for j in peers[0] for i in j]
        peers_column = [k for l in peers[1] for k in l]
        peers_box = [i for j in peers[2] for i in j]
        
        return peers_row, peers_column, peers_box
               
    # Applies the constraints and strategies as long as possible values of cells are still updated.
    def apply_constraints(self):

        empty_cells_before_constraints = len([i for i in self.final_values.reshape(81) if i == 0])
        empty_cells_after_constraints = len([])

        while empty_cells_before_constraints != empty_cells_after_constraints:
            empty_cells_before_constraints = len([i for i in self.final_values.reshape(81) if i == 0])
            self.constraints()
            empty_cells_after_constraints = len([i for i in self.final_values.reshape(81) if i == 0])

        return self.final_values
    
    def is_goal(self):
        
        if all(value != 0 for i in range(9) for value in self.final_values[i]):
            return True
        
        return False

    
    def is_invalid(self):
        
        return any(len(values[i]) == 0  for i in range(9) for values in self.possible_values)


In [2]:
def pick_cell(state,next_cell_candidates):
    
    next_cell = min(next_cell_candidates, key = lambda t: t[0])
    
    min_len_candidates = [next_cell_candidates[i] for i in range(len(next_cell_candidates)) if next_cell_candidates[i][0] == next_cell[0]]

    candidates_and_occurances = []

    for cell in min_len_candidates:
        
        r = cell[1]
        c = cell[2]
        possible_values = cell[3]

        peers =  state.peers_per_unit(r,c)
        peers_unpacked = peers[0]+peers[1]+peers[2]
        total_occurances = 0
        occurances_list = []
        
        for value in possible_values:
            occurances = state.count_occurances(peers_unpacked,value)
            occurances_list.append(occurances)
            total_occurances += occurances
    
        candidates_and_occurances.append((cell,occurances,occurances_list))
    
    # among cells with minimum length choose the one with values that occur maximum times in peers.
    next_cell = max(candidates_and_occurances,key = lambda t: t[1])

    row = next_cell[0][1]
    col = next_cell[0][2]
    next_cell_ordered = [x for y, x in sorted(zip(next_cell[2],next_cell[0][3]), key = lambda t:t[1])]
    
    return row, col, next_cell_ordered
    
         
def search(sudoku):

    if type(sudoku) == tuple:
        state = sudoku_state(*sudoku)
             
    else:
        state = sudoku_state(sudoku)
              
    state.apply_constraints()  
  
    if state.is_invalid() == True: 
        return None
        
    if state.is_goal() == True:
        return state.final_values

    next_cell_candidates = [(len(state.possible_values[row][col]), row, col, state.possible_values[row][col]) for row in range(9) for col in range(9) if len(state.possible_values[row][col]) > 1]
    
    if len(next_cell_candidates) != 0:
        row,col,next_cell = pick_cell(state,next_cell_candidates) 
 
        for num in next_cell:
            new_state = copy.deepcopy(state)
            new_state.update(row,col,num)
            new_state.possible_values[row][col] = [num]
            new_state.final_values[row][col] = num
            new_state.apply_constraints()
  
            if not new_state.is_invalid(): 
                deep_arr = search((new_state.final_values, new_state.possible_values, new_state.final_values))
                
                if deep_arr is not None:           
                    deep_state = sudoku_state(deep_arr,deep_arr, deep_arr)
                    
                    return deep_state.final_values
    return None



def sudoku_solver(sudoku):
    
    solved_sudoku = search(sudoku)
    
    if solved_sudoku is None:
        return np.full(shape = (9,9), fill_value = -1, dtype=np.int)
    else:   
        return solved_sudoku   